##### Импорт и установка необходимых пакетов и модулей

In [1]:
!pip install openpyxl

In [3]:
!pip install pandas

In [106]:
import pandas as pd
import re
import math
from scipy.stats import chi2_contingency 
import matplotlib.pyplot as plt
%matplotlib inline

##### Координаты Кремля (Путин, помоги)

In [107]:
kremlin_long = 37.628286
kremlin_lat = 55.760156

##### Чтение файла

In [108]:
df_food = pd.read_excel('Общественное питание в Москве.xlsx', encoding = 'utf-8')

In [109]:
df_food

,Name,IsNetObject,OperatingCompany,TypeObject,AdmArea,District,Address,PublicPhone,SeatsCount,SocialPrivileges,geoData,global_id
0,СМЕТАНА,нет,NaN,кафе,Северо-Восточный административный округ,Ярославский район,"город Москва, улица Егора Абакумова, дом 9",Контактный телефон: (499) 183-14-10,48,нет,Геоданные: ↴\nКоординаты: [\n 37.714565000436...,637376221
1,Родник,нет,NaN,кафе,Центральный административный округ,Таганский район,"город Москва, улица Талалихина, дом 2/1, корпус 1",Контактный телефон: (495) 676-55-35,35,нет,Геоданные: ↴\nКоординаты: [\n 37.673306130034...,637376331
2,Кафе «Академия»,нет,NaN,кафе,Центральный административный округ,Таганский район,"город Москва, Абельмановская улица, дом 6",Контактный телефон: (495) 662-30-10,95,нет,Геоданные: ↴\nКоординаты: [\n 37.669647596938...,637376349
3,ГБОУ «Школа № 1430 имени Героя Социалистическо...,нет,NaN,столовая,Северо-Восточный административный округ,район Лианозово,"город Москва, Угличская улица, дом 17",Контактный телефон: (499) 908-06-15,240,нет,"Геоданные: ↴\nКоординаты: [\n 37.56694,\n 55...",637376480
4,Брусника,да,Брусника,кафе,Центральный административный округ,район Арбат,"город Москва, переулок Сивцев Вражек, дом 6/2",Контактный телефон: (495) 697-04-89,10,нет,"Геоданные: ↴\nКоординаты: [\n 37.59812754844,...",637376481
...,...,...,...,...,...,...,...,...,...,...,...,...
17220,Суши,нет,NaN,магазин (отдел кулинарии),Восточный административный округ,район Измайлово,"Российская Федерация, город Москва, внутригоро...",Контактный телефон: (925) 195-18-19,0,нет,Геоданные: ↴\nКоординаты: [\n 37.782898134930...,1111677810
17221,Ресторан «Аист»,нет,NaN,ресторан,Центральный административный округ,Пресненский район,"город Москва, Малая Бронная улица, дом 8, стро...",Контактный телефон: (499) 940-70-40,200,нет,Геоданные: ↴\nКоординаты: [\n 37.596924999999...,1111678199
17222,Суши Wok,да,Суши Wok,кафе,Юго-Западный административный округ,район Южное Бутово,"город Москва, Скобелевская улица, дом 19",Контактный телефон: (499) 754-44-44,22,нет,"Геоданные: ↴\nКоординаты: [\n 37.556331,\n 5...",1111678251
17223,Теремок,да,Теремок,предприятие быстрого обслуживания,Южный административный округ,Даниловский район,"город Москва, улица Ленинская Слобода, дом 17",Контактный телефон: нет телефона,48,нет,"Геоданные: ↴\nКоординаты: [\n 37.657186,\n 5...",1111678371


In [110]:
df_geo = df_food["geoData"]
df_geo

0        Геоданные: ↴\nКоординаты: [\n  37.714565000436...
1        Геоданные: ↴\nКоординаты: [\n  37.673306130034...
2        Геоданные: ↴\nКоординаты: [\n  37.669647596938...
3        Геоданные: ↴\nКоординаты: [\n  37.56694,\n  55...
4        Геоданные: ↴\nКоординаты: [\n  37.59812754844,...
                               ...                        
17220    Геоданные: ↴\nКоординаты: [\n  37.782898134930...
17221    Геоданные: ↴\nКоординаты: [\n  37.596924999999...
17222    Геоданные: ↴\nКоординаты: [\n  37.556331,\n  5...
17223    Геоданные: ↴\nКоординаты: [\n  37.657186,\n  5...
17224    Геоданные: ↴\nКоординаты: [\n  37.598553,\n  5...
Name: geoData, Length: 17225, dtype: object

##### Рассчёт расстояния между заведениями и Кремлём
Формула: sqrt((|широта Кремля|-|широта заведения|)^2 + (|долгота Кремля|-|долгота заведения|)^2)

Модуль на каждой координате - если мы вдруг захотим вместо Кремля взять Статую Христа Искупителя в Рио-Де-Жанейро и заведения рассматривать захотим тоже в Рио.

Квадраты и корни - Теорема Пифагора. 

Отличие от реальности - рассчёт в градусах, а не в километрах, погрешности из-за того, что Земля - шар, однако в исследовании нет сравнения с реальными числами, поэтому погрешность нерелевантна

In [111]:
distance_kremlin_all = []
for df_geo_value in df_geo:
    place_coord = re.findall('[0-9]{2}\.*[0-9]*', df_geo_value)
    long_of_kremlin = math.fabs(kremlin_long)-math.fabs(float(place_coord[0]))
    lat_of_kremlin = math.fabs(kremlin_lat)-math.fabs(float(place_coord[1]))
    distance = math.sqrt(long_of_kremlin**2+lat_of_kremlin**2)
    distance_kremlin_all.append(distance)

##### distance = близость от Кремля (в градусах)

In [112]:
seats_all = []
for df_seats in df_food["SeatsCount"]:
    seats_all.append(df_seats)

##### Рассчёт расстояния между заведениями и Кремлём

Мы решили не рассматривать заведения без посадочных мест, а также те заведения, где посадочных мест было мало.

In [114]:
data_distance = [distance_kremlin_all, seats_all]
counter = 0
for seat in data_distance[1]:
    counter+=1
    if seat <= 5:
        seats_all.remove(seats_all[counter])
        distance_kremlin_all.remove(distance_kremlin_all[counter])


##### Применяем χ^2 

Нулевая гипотеза: есть связь между количеством мест в заведении и дальностью от Кремля

In [115]:
chi, p, dof, expected = chi2_contingency(data_distance)
significance_level = [0.01, 0.05]
significance = 0
for level_test in significance_level:
    print('p value for α=' + str(level_test) + ': ' + str(p)) 
    if p <= sign_level: 
        significance += 1
if significance == 2:
    print('Null Hypothesis Is Not Confirmed')
elif significance == 0:
    print ('Null Hypothesis Is Accepted')
else:
    print('Null Hypothesis Is Accepted For α=0.05, Not Confirmed For α=0.01')

p value for α=0.01: 0.0
p value for α=0.05: 0.0
Null Hypothesis Is Not Confirmed


##### Гипотеза не подтвердилась